In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd

# dataframe with rag answers

df = pd.read_csv('../dataset_qa_all_gpt4o_answ.csv', sep='@')
columns = {
    'question': 'Вопрос',
    'golden_answer': 'Ответ',
    'rag_answer': 'gpt4o',
    # 'context_chunks': 'chunks'  # TODO
}

df = pd.DataFrame(data={col : df[col_old] for col, col_old in columns.items()})
df.head()  # we work on evaluation of data in such format

,question,golden_answer,rag_answer
0,Какие требования предъявляются для перехода на...,"Кредитная организация, соответствующая критери...",Для перехода на применение расчетного КВП необ...
1,Какие действия нужно предпринять в случае реор...,При присоединении другой кредитной организации...,При реорганизации в форме присоединения другой...
2,Какие особенности применения расчетного КВП су...,Системно значимые кредитные организации обязан...,Особенности применения расчетного КВП для сист...
3,Какие методики и процедуры расчета оп риска до...,Организация должна зафиксировать во внутренних...,Методики и процедуры расчета операционного рис...
4,Какие требования должны быть выполнены для иск...,Кредитная организация может исключить определе...,Для исключения доходов и расходов из расчета к...


In [ ]:
from tokens import GIGACHAT_AUTHORIZATION_KEY
from langchain.chat_models.gigachat import GigaChat

# insert your token
# GIGACHAT_AUTHORIZATION_KEY = ...

model = GigaChat(
    credentials=GIGACHAT_AUTHORIZATION_KEY,
    scope="GIGACHAT_API_PERS",
    model=["GigaChat", "GigaChat-Pro"][0],
    # Отключает проверку наличия сертификатов НУЦ Минцифры
    verify_ssl_certs=False,
)

model.invoke('say something').content

'Конечно! Что именно ты хочешь, чтобы я сказал?'

In [17]:
from validation_generation import calculate_generation_metrics_v1

df_metrics = calculate_generation_metrics_v1(df, model)
df_metrics.head()

100%|██████████| 40/40 [00:37<00:00,  1.07it/s]


,question,golden_answer,rag_answer,bleu,rouge-1_r,rouge-1_p,rouge-1_f,rouge-2_r,rouge-2_p,rouge-2_f,rouge-l_r,rouge-l_p,rouge-l_f,llm_score1_desc,llm_score1
0,Какие требования предъявляются для перехода на...,"Кредитная организация, соответствующая критери...",Для перехода на применение расчетного КВП необ...,0.014105,0.169014,0.387097,0.235294,0.086420,0.205882,0.121739,0.154930,0.354839,0.215686,The model answer is somewhat relevant but lack...,6
1,Какие действия нужно предпринять в случае реор...,При присоединении другой кредитной организации...,При реорганизации в форме присоединения другой...,0.018000,0.175439,0.322581,0.227273,0.048387,0.093750,0.063830,0.175439,0.322581,0.227273,The model answer is largely consistent with th...,8
2,Какие особенности применения расчетного КВП су...,Системно значимые кредитные организации обязан...,Особенности применения расчетного КВП для сист...,0.009707,0.125000,0.166667,0.142857,0.019231,0.026316,0.022222,0.125000,0.166667,0.142857,The model answer provides a general overview b...,5
3,Какие методики и процедуры расчета оп риска до...,Организация должна зафиксировать во внутренних...,Методики и процедуры расчета операционного рис...,0.078697,0.194444,0.233333,0.212121,0.095238,0.121212,0.106667,0.166667,0.200000,0.181818,The model answer is partially relevant and som...,6
4,Какие требования должны быть выполнены для иск...,Кредитная организация может исключить определе...,Для исключения доходов и расходов из расчета к...,0.009645,0.254545,0.437500,0.321839,0.016667,0.031250,0.021739,0.200000,0.343750,0.252874,The model's answer is somewhat relevant but la...,7


In [18]:
df_metrics.drop(columns=['question', 'golden_answer', 'rag_answer', 'llm_score1_desc']).mean(axis=0)

bleu          0.037931
rouge-1_r     0.185295
rouge-1_p     0.284700
rouge-1_f     0.222548
rouge-2_r     0.066529
rouge-2_p     0.109082
rouge-2_f     0.081815
rouge-l_r     0.159749
rouge-l_p     0.246427
rouge-l_f     0.192146
llm_score1    5.900000
dtype: float64